# Code Converter - Python to C++

In [128]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [129]:
# environment

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')


In [130]:
# initialize
# NOTE - option to use ultra-low cost models by uncommenting last 2 lines

openai = OpenAI()
deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

OPENAI_MODEL = "gpt-4o"
DEEPSEEK_MODEL="deepseek-chat"


In [131]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an Windows 11. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [132]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [133]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [134]:
# write to a file called optimized.cpp

def write_output(cpp,*model):
    code = cpp.replace("```cpp","").replace("```","")
    if model == 'gpt':
        with open("optimized_gpt.cpp", "w") as f:
            f.write(code)
    elif model == 'deepseek':
        with open("optimized_deepseek.cpp", "w") as f:
            f.write(code)
    else:
        with open("optimized.cpp", "w") as f:
            f.write(code)

In [135]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply,'gpt')

In [136]:
def optimize_deepseek(python):
    result = deepseek_via_openai_client.chat.completions.create(
        model=DEEPSEEK_MODEL,
        messages=messages_for(python),
        stream=True
    )
    reply = ""
    for chunk in result:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply,'deepseek')

In [137]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [12]:
exec(pi)

Result: 3.141592658589
Execution Time: 10.601091 seconds


In [13]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

 Function to perform the calculation
double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        // Perform arithmetic operations and update result
        double j = static_cast<double>(i * param1 - param2);
        result -= (1.0 / j);
        j = static_cast<double>(i * param1 + param2);
        result += (1.0 / j);
    }
    return result;
}

int main() {
    // Start time measurement
    auto start_time = std::chrono::high_resolution_clock::now();
    
    // Perform calculation
    double result = calculate(100000000, 4, 1) * 4;
    
    // End time measurement
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_seconds = end_time - start_time;

    // Output the result with precision
    std::cout << "Result: " << std::setprecision(12) << result << std::endl;
    std::cout << "Execution T

In [14]:
exec(pi)

Result: 3.141592658589
Execution Time: 10.638783 seconds


In [15]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        int j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    using namespace std::chrono;
    auto start_time = high_resolution_clock::now();

    double result = calculate(100'000'000, 4, 1) * 4;

    auto end_time = high_resolution_clock::now();
    duration<double> execution_time = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12)
              << "Result: " << result << std::endl
              << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [16]:
# Compile C++ and run the executable

! g++ ./optimized_gpt.cpp -o optimized_gpt
! ./optimized_gpt


Result: 3.141592658589
Execution Time: 0.385336000000 seconds


In [17]:
optimize_deepseek(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << elapsed.count() << " seconds" << std::endl;

    return 0;
}
```

In [18]:
! g++ ./optimized_deepseek.cpp -o optimized_deepseek
! ./optimized_deepseek

Result: 3.141592658589
Execution Time: 0.400021000000 seconds


In [138]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [139]:
def stream_deepseek(python):    
    stream = deepseek_via_openai_client.chat.completions.create(model=DEEPSEEK_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [140]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Deepseek":
        result = stream_deepseek(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far      

In [16]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Deepseek"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [141]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [142]:
def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["g++", "./optimized.cpp", "-o", "optimized"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [143]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [20]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Deepseek"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [160]:
from huggingface_hub import login, InferenceClient
from transformers import TextStreamer, pipeline,AutoTokenizer, AutoModelForCausalLM,TextIteratorStreamer
from threading import Thread
import torch

In [145]:
hf_token = os.environ['HUGGINGFACE_RW_TOKEN']
login(hf_token, add_to_git_credential=True)

In [161]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/CodeQwen1.5-7B-Chat")
Qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/CodeQwen1.5-7B-Chat",
    torch_dtype=torch.float16,
    device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [162]:
messages = messages_for(pi)
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [164]:
tokenizer.pad_token = tokenizer.eos_token
model_inputs = tokenizer([text], return_tensors="pt").to('cuda')


In [165]:
if 'token_type_ids' in model_inputs:
    del model_inputs['token_type_ids']

In [166]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [167]:
Qwen_model.generate(
    **model_inputs,
    streamer=streamer,
    max_new_tokens=300,
    do_sample=False,
    temperature=0.7
)

Here is the C++ code that reimplements the Python code:

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        int j = i * param1 - param2;
        result -= 1.0 / j;
        j = i * param1 + param2;
        result += 1.0 / j;
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << "Result: " << std::setprecision(12) << result << std::endl;
    std::cout << "Execution Time: " << std::chrono::duration<double>(end_time - start_time).count() << " seconds" << std::endl;

    return 0;
}
```

This C++ code does the same thing as the Python code: it calc


tensor([[    3, 10326,  1396,  4260,  1789,  1748, 13710,  1726,  1692,  1698,
         16784, 18773,  4214,  1672,  2522,  4187,  1718,  4778,  1713,  1748,
          8433, 35321,    39, 35342, 28432,  2151,  1749,  1718,  4778,  4214,
         35382,  2243,  5702,  1840,  3448,  1716,  1679,  1858,  1821,  3403,
          2050, 12160,  2037,  2096, 19236,  5702, 35342,  1751,  1718,  4778,
          4630,  3945,  1676,  6496,  1748, 13433,  5723,  1672,  1651, 18455,
          3335,  2038, 35342,     4,  1396,     3,  5435,  1396, 35368,  2182,
          5334,  1820, 18773,  4214,  1672,  1718,  4778,  1749,  1651, 18455,
          3335,  9423,  1726, 13321, 13433,  5723,  1672,  1651,  3373,  2038,
         35342, 28432,  2151,  1749,  1718,  4778,  4214, 35382,  1858,  1821,
          7565,  1902,  2095,  2037,  2096,  1643,  2780,  5702, 35342,  9863,
          5018,  1676,  2697,  5484,  1676,  5532,  1968,  2308,  2036, 28530,
         35342, 12869,  1676,  2184,  4922,  1857,  

In [ ]:
generated_ids = Qwen_model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
] 

In [71]:
response = tokenizer.batch_decode(model_inputs, skip_special_tokens=True)[0]

In [72]:
response

'Here is the C++ code that reimplements the Python code:\n\n```cpp\n#include <iostream>\n#include <iomanip>\n#include <chrono>\n\ndouble calculate(int iterations, int param1, int param2) {\n    double result = 1.0;\n    for (int i = 1; i <= iterations; ++i) {\n        int j = i * param1 - param2;\n        result -= 1.0 / j;\n        j = i * param1 + param2;\n        result += 1.0 / j;\n    }\n    return result;\n}\n\nint main() {\n    auto start_time = std::chrono::high_resolution_clock::now();\n    double result = calculate(100000000, 4, 1) * 4;\n    auto end_time = std::chrono::high_resolution_clock::now();\n\n    std::cout << "Result: " << std::setprecision(12) << result << std::endl;\n    std::cout << "Execution Time: " << std::chrono::duration<double>(end_time - start_time).count() << " seconds" << std::endl;\n\n    return 0;\n}\n```\n\nThis C++ code does the same thing as the Python code: it calculates a mathematical function and measures the execution time. The main differences 